# 라이브러리

In [ ]:
import pandas as pd
import numpy as np
import re 

from tqdm.auto import tqdm
from joblib import parallel_backend
import contextlib


In [ ]:
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    from joblib import Parallel
    old_batch_callback = Parallel.BatchCompletionCallBack

    class TqdmBatchCompletionCallBack(old_batch_callback):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    Parallel.BatchCompletionCallBack = TqdmBatchCompletionCallBack
    try:
        yield tqdm_object
    finally:
        Parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()


# 재무정보_final_v2(16~24) 파일 열기
- 기업 정보만을 위해 열어보는 파일
    - 재무 데이터에 등장한 모든 기업
    - 연도별로 살아 있었던 기업
    - 특허 여부와 무관

In [ ]:
financial_df = pd.read_csv('../../data/재무정보_final_v2(16~24)_수정_최종.csv', encoding="utf-8", encoding_errors='replace', engine="python")
financial_df.tail(5)

In [ ]:
financial_df.columns.tolist()

# 특허정보_final 파일 열기

In [ ]:
patent_final = pd.read_csv('../../data/특허정보_final_v2.csv', encoding="cp949", encoding_errors='replace', engine="python")
patent_final.head(5)

In [ ]:
patent_final.columns.tolist()

In [ ]:
patent_final = patent_final.rename(columns={
    'ipcNumber_IPC코드': 'ipcNumber',
    'applicationDate_출원일자': 'applicationDate',
    'astrtCont_초록':'astrtCont',
    'applicationNumber_출원번호':'applicationNumber',
     'registerDate_등록일자':'registerDate',
      'indexNo_일련번호':'indexNo',
 'registerStatus_등록상태':'registerStatus',
 'inventionTitle_발명의명칭':'inventionTitle',
 'applicantName_출원인명':'applicantName'



    
})


# 잠깐 .......

## 특허 데이터 최소 전처리

In [ ]:
patent_df = patent_final.copy()

# 출원일 datetime 보장
patent_df['applicationDate'] = pd.to_datetime(patent_df['applicationDate'])

# 출원년도 
patent_df['application_year'] = patent_df['applicationDate'].dt.year


In [ ]:
patent_df['application_year'] = pd.to_numeric(
    patent_df['application_year'],
    errors='coerce'
)


In [ ]:
def normalize_company_name(name: str) -> str:
    if pd.isna(name):
        return None

    name = str(name)

    # 1. 소문자 변환
    name = name.lower()

    # 2. 법인 표기 제거
    name = re.sub(r'\(주\)|㈜|주식회사', '', name)

    # 3. 불필요한 공백 제거
    name = re.sub(r'\s+', '', name)

    # 4. 특수문자 제거 (필요 최소)
    name = re.sub(r'[^\w가-힣]', '', name)

    return name


In [ ]:
#=====================================================================================
# 두 데이터에 동일하게 적용 
#=====================================================================================
financial_df['company_norm'] = financial_df['기업명'].apply(normalize_company_name)
patent_df['company_norm'] = patent_df['company_name'].apply(normalize_company_name)

#=====================================================================================
# 매칭 품질 진단
#=====================================================================================

# 재무 기준 기업 수
financial_companies = set(financial_df['company_norm'].dropna())

# 특허 기준 기업 수
patent_companies = set(patent_df['company_norm'].dropna())

print("재무 기업 수:", len(financial_companies))
print("특허 기업 수:", len(patent_companies))
print("교집합 수:", len(financial_companies & patent_companies))


In [ ]:
company_year_df = (
    financial_df[['company_norm', '연도']]
    .drop_duplicates()
    .sort_values(['company_norm', '연도'])
    .reset_index(drop=True)
)


In [ ]:
# =====================================================
# [STEP 0] 매출액 숫자 변환 (반드시 growth 계산 전에)
# =====================================================

financial_df['매출액'] = (
    financial_df['매출액']
    .astype(str)                 # 혹시 모를 타입 통일
    .str.replace(',', '')        # 천 단위 콤마 제거
    .str.replace(' ', '')        # 공백 제거
)

financial_df['매출액'] = pd.to_numeric(
    financial_df['매출액'],
    errors='coerce'              # 숫자 아닌 건 NaN
)

# =====================================================
# [STEP 1] 정렬 (절대 생략 금지)
# =====================================================
financial_df = financial_df.sort_values(
    ['company_norm', '연도']
)

# =====================================================
# [STEP 2] 전년도 매출 (lag)
# =====================================================
financial_df['revenue_lag_1'] = (
    financial_df
    .groupby('company_norm')['매출액']
    .shift(1)
)

# =====================================================
# [STEP 3] 매출 성장률
# =====================================================
financial_df['revenue_growth'] = (
    (financial_df['매출액'] - financial_df['revenue_lag_1'])
    / financial_df['revenue_lag_1']
)
# =====================================================
# [STEP 4] 성장 여부 라벨
# =====================================================
financial_df['growth_flag'] = (
    financial_df['revenue_growth'] > 0
).astype(int)

# =====================================================
# [STEP 5] 최종 타겟 변수
# =====================================================
financial_df['y'] = financial_df['growth_flag']


## 특허 피처 생성 함수

In [ ]:
import pandas as pd
import numpy as np

def make_patent_features(company_norm, t, patent_df):
    """
    company_norm × 연도(t) 기준 특허 피처 생성
    - warm-up 적용: t < 2019 → None
    - 누수 방지: t-1 시점까지만 사용
    """

    # ===============================
    # 0. Warm-up 강제
    # ===============================
    t = int(t)
    if t < 2019:
        return None

    # 기업 기준 필터
    df = patent_df[patent_df['company_norm'] == company_norm].copy()

    # ===============================
    # 1. 기간 필터 (누수 방지 핵심)
    # ===============================
    window_5y = df[
        (df['application_year'] >= t - 4) &
        (df['application_year'] <= t - 1)
    ]

    window_recent = df[
        (df['application_year'] >= t - 2) &
        (df['application_year'] <= t - 1)
    ]

    # # 등록률 코호트: t-4 ~ t-3
    # cohort_register = df[
    #     (df['application_year'] >= t - 4) &
    #     (df['application_year'] <= t - 3)
    # ]

    # 피인용 as-of cutoff
    citation_window = window_5y[
        window_5y['applicationDate'] <= pd.Timestamp(f"{t-1}-12-31")
    ].copy()

    # ===============================
    # 2. 피처 계산
    # ===============================
    patent_count_5y = len(window_5y)
    patent_count_recent = len(window_recent)

    ipc_diversity = (
        window_5y['ipcNumber']
        .dropna()
        .astype(str)
        .str[:4]
        .nunique()
        if patent_count_5y > 0 else 0
    )

    # # 등록률: registerDate 기준
    # if len(cohort_register) > 0:
    #     patent_register_rate = (
    #         cohort_register['registerDate'].notna().mean()
    #     )
    # else:
    #     patent_register_rate = 0.0
    # ===============================
    # 등록률 코호트: t-4 ~ t-3
    # ===============================
    VALID_SUCCESS_STATUS = ['등록', '공개']
    cohort_register = df[
        (df['application_year'] >= t - 4) &
        (df['application_year'] <= t - 3)
    ]

    if len(cohort_register) > 0:
        # 전체 출원 수
        total_applications = len(cohort_register)

        # 등록 + 공개 건수
        success_count = cohort_register[
            cohort_register['registerStatus'].isin(['등록', '공개'])
        ].shape[0]

        patent_register_rate = success_count / total_applications
    else:
        patent_register_rate = 0.0


    # -------------------------------
    # 피인용 (기본)
    # -------------------------------
    patent_citation_total = (
        citation_window['피인용 횟수'].sum()
        if len(citation_window) > 0 else 0.0
    )

    patent_citation_avg = (
        patent_citation_total / max(1, patent_count_5y)
    )

    # -------------------------------
    # 🔥 연령 보정 피인용
    # -------------------------------
    if len(citation_window) > 0:
        citation_window['patent_age'] = (
            t - citation_window['application_year']
        ).clip(lower=1)

        citation_window['citation_per_year'] = (
            citation_window['피인용 횟수'] / citation_window['patent_age']
        )

        patent_citation_age_adj_total = (
            citation_window['citation_per_year'].sum()
        )
    else:
        patent_citation_age_adj_total = 0.0

    patent_citation_age_adj_avg = (
        patent_citation_age_adj_total / max(1, patent_count_5y)
    )

    # -------------------------------
    # 특허 0건 플래그
    # -------------------------------
    no_patent_flag = int(patent_count_5y == 0)

    return {
        'company_norm': company_norm,
        '연도': t,
        'patent_count_5y': patent_count_5y,
        'patent_count_recent': patent_count_recent,
        'ipc_diversity': ipc_diversity,
        'patent_register_rate': patent_register_rate,
        'patent_citation_total': patent_citation_total,
        'patent_citation_avg': patent_citation_avg,
        # 🔥 추가 피처
        'patent_citation_age_adj_total': patent_citation_age_adj_total,
        'patent_citation_age_adj_avg': patent_citation_age_adj_avg,
        'no_patent_flag': no_patent_flag
    }


In [ ]:
patent_feature_rows = []

for row in tqdm(
    company_year_df.itertuples(index=False),
    total=len(company_year_df),
    desc="특허 피처 생성"
):
    feat = make_patent_features(
        company_norm=row.company_norm,
        t=row.연도,
        patent_df=patent_df
    )

    if feat is not None:
        patent_feature_rows.append(feat)

patent_feature_df = pd.DataFrame(patent_feature_rows)


In [ ]:
patent_feature_df['연도'].unique()


- 특허 피처 엔지니어링 과정에서 롤링 윈도우의 warm-up 기간을 함수 내부에서 강제하여,
충분한 과거 데이터가 확보되지 않은 연도(2019 이전)가
학습 데이터에 포함되지 않도록 수정하였다.

## 재무 데이터와 결합

In [ ]:
financial_df['연도'] = pd.to_numeric(financial_df['연도'], errors='coerce').astype('Int64')
patent_feature_df['연도'] = pd.to_numeric(patent_feature_df['연도'], errors='coerce').astype('Int64')


In [ ]:
financial_df[['연도']].dtypes, patent_feature_df[['연도']].dtypes


In [ ]:
financial_df['매출액'].isna().mean(), (financial_df['매출액'] == 0).mean()


In [ ]:
financial_agg = (
    financial_df
    .groupby(['company_norm', '연도'], as_index=False)
    .agg({
        '기업명': 'first',
        '사업자등록번호': 'first',
        'crno': 'first',
        'dart_corp_code': 'first',
        'sectors': 'first',
        'field': 'first',

        '매출액': 'sum',
        '영업이익': 'sum',
        '당기순이익': 'sum',
        '자산총계': 'max',
        '부채총계': 'max',
        '자본총계': 'max',
        '연구개발비': 'sum',
        'CAPEX': 'sum',
        '유형자산_당기': 'max',
        '유형자산_전기': 'max',
        '유형자산_증감': 'sum'
    })
)


In [ ]:
final_df = financial_agg.merge(
    patent_feature_df,
    on=['company_norm', '연도'],
    how='left',
    validate='one_to_one'
)


In [ ]:
len(financial_agg), len(final_df)


In [ ]:
final_df['매출액'].isna().mean()


In [ ]:
final_df

In [ ]:
print(financial_df.columns)
print(patent_feature_df.columns)
print(final_df.columns)


In [ ]:
print(len(financial_df), len(final_df))


## 타깃(y) 생성 — “t 정보로 t+1 성장 여부 예측”

In [ ]:
import numpy as np
import pandas as pd

money_cols = [
    '매출액','영업이익','당기순이익',
    '자산총계','부채총계','자본총계',
    '연구개발비','CAPEX',
    '유형자산_당기','유형자산_전기','유형자산_증감'
]

for col in money_cols:
    final_df[col] = (
        final_df[col]
        .astype(str)
        .str.replace(',', '', regex=False)
    )
    final_df[col] = pd.to_numeric(final_df[col], errors='coerce')



In [ ]:
final_df[['매출액']].head()
final_df['매출액'].isna().mean()


In [ ]:
final_df['revenue_growth'] = (
    final_df
    .groupby('company_norm')['매출액']
    .pct_change()
)


In [ ]:
final_df

In [ ]:
final_df = final_df[
    (final_df['연도'] >= 2019) &
    (final_df['연도'] <= 2024)
].copy()


In [ ]:
final_df['연도'].value_counts().sort_index()


In [ ]:
final_df['growth_flag'] = (
    final_df
    .groupby('연도')['revenue_growth']
    .transform(lambda x: (x >= x.quantile(0.7)).astype(int))
)


In [ ]:
final_df.groupby('연도')['growth_flag'].mean()


In [ ]:
final_df['y'] = (
    final_df
    .groupby('company_norm')['growth_flag']
    .shift(-1)
)


In [ ]:
model_df = final_df.dropna(subset=['y']).copy()
model_df['y'] = model_df['y'].astype(int)


In [ ]:
model_df['연도'].value_counts().sort_index()


In [ ]:
model_df

In [ ]:
# 필요한 컬럼들만 남기기
keep_cols = [
    # 식별 / 시점
    'company_norm',
    '연도',
    '기업명',

    # 특허 피처
    'patent_count_5y',
    'patent_count_recent',
    'ipc_diversity',
    'patent_register_rate',
    'patent_citation_total',
    'patent_citation_avg',
    'patent_citation_age_adj_total',
    'patent_citation_age_adj_avg',
    'no_patent_flag',

    # 성장 관련
    'revenue_growth',
    'growth_flag',
    'y'
]

model_df = model_df[keep_cols].copy()


In [ ]:
model_df.to_csv('../../output/feature_engineering_dataset_7_patent.csv', encoding='utf-8-sig')

# 일단 .. 대기 (폐기)

### EDA

In [ ]:
# EDA 대상 컬럼 정리
patent_cols = [
    'patent_count_5y',
    'patent_count_recent',
    'ipc_diversity',
    'patent_register_rate',
    'patent_citation_total',
    'patent_citation_avg',
    'patent_citation_age_adj_total',
    'patent_citation_age_adj_avg',
    'no_patent_flag'
]


In [ ]:
final_df[patent_cols].describe()


In [ ]:
final_df['no_patent_flag'].value_counts()


In [ ]:
final_df.groupby('no_patent_flag')[patent_cols].mean()

In [ ]:
final_df[['patent_count_5y', 'patent_count_recent']].corr()

In [ ]:
# 특허를 많이 내는 회사일수록 IPC도 더 다양해지는 경향이 있음
final_df[['patent_count_5y', 'ipc_diversity']].corr()


In [ ]:
# 정규화 비율 피처
# : 규모 효과(특허 수)를 제거하고 구조적 차이를 보기 위한 변수
# 같은 특허 수 대비 기술 폭을 알려줌
final_df['ipc_diversity_ratio'] = (
    final_df['ipc_diversity'] / final_df['patent_count_5y'].replace(0, 1)
)

final_df[['patent_count_5y', 'ipc_diversity_ratio']].corr()


In [ ]:
final_df['patent_register_rate'].describe()

# 저분산 변수 -> 대부분의 값이 같아서 예측에 기여하지 못하는 변수
# 등록률이 아닌 '등록 건수 기반'으로 변경 -> “얼마나 등록되었는가”를 직접 표현

In [ ]:
final_df['registered_patent_count'] = (
    final_df['patent_register_rate'] * final_df['patent_count_5y']
)


In [ ]:
final_df.to_csv('../../output/feature_engineering_3.csv', encoding='utf-8-sig')

In [ ]:
final_df['no_patent_flag'].value_counts(normalize=True)


### 시각화

In [ ]:
## 기본 세팅
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['axes.grid'] = True

In [ ]:
## 특허 활동량 분포
# - 특허는 일부 기업에 집중되어 있음
plt.hist(final_df['patent_count_5y'], bins=50)
plt.title('Distribution of Patent Count (5Y)')
plt.xlabel('patent_count_5y')
plt.ylabel('Frequency')
plt.show()


In [ ]:
## 최근성 vs 누적 활동량

plt.scatter(
    final_df['patent_count_5y'],
    final_df['patent_count_recent'],
    alpha=0.3
)
plt.xlabel('patent_count_5y')
plt.ylabel('patent_count_recent')
plt.title('Recent vs Total Patent Activity')
plt.show()


In [ ]:
# 👉 “ipc_diversity는 특허 수 복사인가?” 검증
plt.scatter(
    final_df['patent_count_5y'],
    final_df['ipc_diversity'],
    alpha=0.3
)
plt.xlabel('patent_count_5y')
plt.ylabel('ipc_diversity')
plt.title('Patent Count vs IPC Diversity')
plt.show()


In [ ]:
# 등록률 분포 (문제 피처 확인용)
# 👉 “왜 이 피처를 대체해야 했는지” 시각적으로 증명
plt.hist(final_df['patent_register_rate'], bins=20)
plt.title('Patent Register Rate Distribution')
plt.xlabel('patent_register_rate')
plt.show()


In [ ]:
plt.hist(final_df['patent_citation_total'], bins=50)
plt.title('Total Citation Count')
plt.xlabel('patent_citation_total')
plt.show()


In [ ]:
plt.hist(final_df['patent_citation_age_adj_total'], bins=50)
plt.title('Age-adjusted Citation Count')
plt.xlabel('patent_citation_age_adj_total')
plt.show()


In [ ]:
# 특허 보유 vs 미보유 기업 비교

In [ ]:
group_means = (
    final_df
    .groupby('no_patent_flag')[
        ['patent_count_5y', 'patent_citation_total']
    ]
    .mean()
)

group_means.plot(kind='bar')
plt.title('Patent vs No-Patent Firms')
plt.xticks(rotation=0)
plt.show()


# 모델링
- 모델 : LogReg, RandomForest, LightBGM, XGBoost
- 검증 방식 : Holdout, GroupKFold, RollingValidation
- 평가지표: PR-AUC, Top-20/50 Precision, F1, Brier
---
> - model_df : feature + y 포함 (정상)
> - 연도 범위 : 2019–2023
> - y : t년 피처 → t+1년 성장 여부(0,1)

## Feature / Label 분리

In [ ]:
# ID / Label 컬럼
id_cols = ['기업명', 'company_norm', '연도']
label_cols = ['y', 'growth_flag', 'revenue_growth', 'asset_growth']

# 숫자형 피처만 사용
feature_cols = (
    model_df
    .drop(columns=id_cols + label_cols, errors='ignore')
    .select_dtypes(include=['int64', 'float64'])
    .columns
    .tolist()
)

X = model_df[feature_cols].copy()
y = model_df['y'].copy()
years = model_df['연도'].copy()
groups = model_df['company_norm'].copy()


## 결측치 처리 함수

In [ ]:
def fill_na_train_test(X_tr, X_te, feature_cols):
    patent_cols = [c for c in feature_cols if c.startswith('patent_') or c == 'no_patent_flag']
    financial_cols = [c for c in feature_cols if c not in patent_cols]

    # 특허 → 0
    X_tr[patent_cols] = X_tr[patent_cols].fillna(0)
    X_te[patent_cols] = X_te[patent_cols].fillna(0)

    # 재무 → train median
    for col in financial_cols:
        med = X_tr[col].median()
        X_tr[col] = X_tr[col].fillna(med)
        X_te[col] = X_te[col].fillna(med)

    return X_tr, X_te


## 모델 정의

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb

models = {
    "LogReg": LogisticRegression(max_iter=1000),
    "RF": RandomForestClassifier(
        n_estimators=300, max_depth=8, random_state=42, n_jobs=-1
    ),
    "LGBM": lgb.LGBMClassifier(
        n_estimators=300, learning_rate=0.05, random_state=42
    ),
    "XGB": xgb.XGBClassifier(
        n_estimators=300, learning_rate=0.05, max_depth=6,
        eval_metric="logloss", random_state=42
    )
}


## 평가 지표

In [ ]:
import numpy as np
from sklearn.metrics import (
    accuracy_score, roc_auc_score, f1_score,
    average_precision_score, brier_score_loss
)

def top_k_precision(y_true, y_prob, k):
    k = min(k, len(y_true))
    idx = np.argsort(y_prob)[-k:]
    return y_true.iloc[idx].mean()


## 검증

### Hold-Out 
> - Train: 2019–2022
> - Test : 2023 (→ 2024 성장 여부)

In [ ]:
from tqdm import tqdm

results = []

X_tr = X[years <= 2022].copy()
y_tr = y[years <= 2022]

X_te = X[years == 2023].copy()
y_te = y[years == 2023]

X_tr, X_te = fill_na_train_test(X_tr, X_te, feature_cols)

for name, model in tqdm(models.items(), desc="Hold-out"):
    model.fit(X_tr, y_tr)

    prob = model.predict_proba(X_te)[:, 1]
    pred = (prob >= 0.5).astype(int)

    results.append({
        "model": name,
        "validation": "Holdout",
        "Accuracy": accuracy_score(y_te, pred),
        "ROC_AUC": roc_auc_score(y_te, prob),
        "PR_AUC": average_precision_score(y_te, prob),
        "Top20_P": top_k_precision(y_te, prob, 20),
        "Top50_P": top_k_precision(y_te, prob, 50),
        "F1": f1_score(y_te, pred),
        "Brier": brier_score_loss(y_te, prob)
    })


### GroupKFold 검증
- 같은 기업이 Train / Test 에 동시에 등장 x

In [ ]:
from sklearn.model_selection import GroupKFold

gkf = GroupKFold(n_splits=5)

for name, model in tqdm(models.items(), desc="GroupKFold"):
    scores = []

    for tr_idx, te_idx in gkf.split(X, y, groups):
        X_tr, X_te = X.iloc[tr_idx].copy(), X.iloc[te_idx].copy()
        y_tr, y_te = y.iloc[tr_idx], y.iloc[te_idx]

        X_tr, X_te = fill_na_train_test(X_tr, X_te, feature_cols)

        model.fit(X_tr, y_tr)
        prob = model.predict_proba(X_te)[:, 1]
        pred = (prob >= 0.5).astype(int)

        scores.append({
            "Accuracy": accuracy_score(y_te, pred),
            "ROC_AUC": roc_auc_score(y_te, prob),
            "PR_AUC": average_precision_score(y_te, prob),
            "Top20_P": top_k_precision(y_te, prob, 20),
            "Top50_P": top_k_precision(y_te, prob, 50),
            "F1": f1_score(y_te, pred),
            "Brier": brier_score_loss(y_te, prob)
        })

    avg = pd.DataFrame(scores).mean().to_dict()
    avg.update({"model": name, "validation": "GroupKFold"})
    results.append(avg)


### Rolling Validation

In [ ]:
rolling_years = [2021, 2022, 2023]

for name, model in tqdm(models.items(), desc="Rolling"):
    scores = []

    for test_year in rolling_years:
        X_tr = X[years < test_year].copy()
        y_tr = y[years < test_year]

        X_te = X[years == test_year].copy()
        y_te = y[years == test_year]

        X_tr, X_te = fill_na_train_test(X_tr, X_te, feature_cols)

        model.fit(X_tr, y_tr)
        prob = model.predict_proba(X_te)[:, 1]
        pred = (prob >= 0.5).astype(int)

        scores.append({
            "Accuracy": accuracy_score(y_te, pred),
            "ROC_AUC": roc_auc_score(y_te, prob),
            "PR_AUC": average_precision_score(y_te, prob),
            "Top20_P": top_k_precision(y_te, prob, 20),
            "Top50_P": top_k_precision(y_te, prob, 50),
            "F1": f1_score(y_te, pred),
            "Brier": brier_score_loss(y_te, prob)
        })

    avg = pd.DataFrame(scores).mean().to_dict()
    avg.update({"model": name, "validation": "Rolling"})
    results.append(avg)


In [ ]:
results_df = pd.DataFrame(results)
results_df.sort_values(['validation', 'PR_AUC'], ascending=[True, False])
